<a href="https://colab.research.google.com/github/ofonime41/Malaria-Detection-/blob/main/MALARIA_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Malaria Detection ** using TensorFlow.
Dataset is gotten from: https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

In [1]:
!pip install tensorflow

In [2]:
# !pip uninstall keras

In [3]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [5]:
# Get PATH OF THE SATA DIRECTORY
train_dir = '/content/drive/MyDrive/Colab Notebooks/cell_images/Train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/cell_images/Test'

In [6]:
# IMAGINE RESIZING
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 500 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


In [8]:
# Load EfficientNetB0 without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [10]:
# Create full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 234s 14s/step - accuracy: 0.6446 - loss: 0.8840 - val_accuracy: 0.9160 - val_loss: 0.1949
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.8857 - loss: 0.2870 - val_accuracy: 0.9640 - val_loss: 0.1254
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 62s 4s/step - accuracy: 0.9111 - loss: 0.2427 - val_accuracy: 0.9680 - val_loss: 0.1184
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.9405 - loss: 0.1800 - val_accuracy: 0.9480 - val_loss: 0.1560
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.8973 - loss: 0.2364 - val_accuracy: 0.9600 - val_loss: 0.1325
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.9047 - loss: 0.2175 - val_accuracy: 0.9520 - val_loss: 0.1432
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.9045 - loss: 0.2697 - val_accuracy: 0.9600 - val_loss: 0.1159
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.9477 - loss: 0.1517 - val_accuracy: 0.9680 - val_los

In [14]:
# Save trained model
model.save('efficientnetb0_model.keras')